<a href="https://colab.research.google.com/github/maxiperezc/memoscopio/blob/master/HPC/PerezCotoMaximiliano_Ejercicio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# 1 Introducción

El siguiente cuaderno realiza la norma de N de 3 dimensiones vectores en forma secuencial,  utilizando el procesador CPU.

<left>$V_0 = (X_0, Y_0, Z_0)$</left>  
<left>$...$</left>

<left>$V_n = (X_n, Y_n, Z_n)$</left>  

<left>Luego,</left>

<left>$||V_n|| = \sqrt( X^2_n + Y^2_n + Z^2_n)  $</left>  

El objetivo es calcular la fuerza correspondiente a N vectores para ser utilizado en ambientes de desarrollo gráfico o con fines de investigación científica.

---
# 2 Armado del ambiente
No son necesarios, ejecuciones previas del armado del ambiente.

# 3 Desarrollo

In [ ]:
# --------------------------------------------
#@title 3.1 Parámetro de ejecución: cantidad de vectores de 3 dimensiones { vertical-output: true }

N = 5#@param {type: "number"}

# --------------------------------------------

from datetime import datetime
import numpy

tiempo_total = datetime.now()


# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0


# --------------------------------------------
# CPU - Defino la memoria de los vectores en cpu.
x_cpu = numpy.random.randn( cantidad_N )
x_cpu = x_cpu.astype( numpy.float32() )

y_cpu = numpy.random.randn( cantidad_N )
y_cpu = y_cpu.astype( numpy.float32() )

z_cpu = numpy.random.randn( cantidad_N )
z_cpu = y_cpu.astype( numpy.float32() )

v_cpu = numpy.empty_like( x_cpu )



tiempo_bucle = datetime.now()

for idx in range( 0, cantidad_N ):
  v_cpu[idx] = numpy.sqrt(pow(x_cpu[idx], 2) + pow(y_cpu[idx], 2) + pow(z_cpu[idx], 2))

tiempo_bucle = datetime.now() - tiempo_bucle

# --------------------------------------------


# CPU - Informo el resultado.
print( "------------------------------------")
print( "X: " )
print( x_cpu )
print( "------------------------------------")
print( "Y: " )
print( y_cpu )
print( "------------------------------------")
print( "Z: " )
print( z_cpu )
print( "------------------------------------")

print( "V: " )
print( v_cpu )
print( "------------------------------------")


tiempo_total = datetime.now() - tiempo_total

print("Tiempo Total: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo bucle: ", tiempo_en_ms( tiempo_bucle ), "[ms]" )

---
#4 Tabla de pasos
Tabla de pasos de la ejecución del programa:

 Procesador | Función | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo inicial.
CPU      |  numpy.random.randn( Cantidad_N ) | Inicializa los vectores A, B y R.
CPU      |  for...                | Realiza la norma de los vectores $(X_0, Y_0, Z_0) ... (X_n, Y_n, Z_n) $ , guardando el resultado en $V_n$. 
CPU      |  datetime.now()        | Toma el tiempo final.
CPU      |  print()               | Informa los resultados.



---
# 5 Conclusiones

Las conclusiones son explicadas en clase ....

---
# 6 Bibliografía

[1] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Función Axpy de biblioteca BLAS: [Referencia](https://software.intel.com/content/www/us/en/develop/documentation/mkl-developer-reference-c/top/blas-and-sparse-blas-routines/blas-routines/blas-level-1-routines-and-functions/cblas-axpy.html)

[4] Biblioteca BLAS: [Referencia](http://www.netlib.org/blas/)

[5] Tutorial Point Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

---
# GPU

---
# 2 Armado del ambiente
Instala en el cuaderno el módulo CUDA de Python.

# 3 Desarrollo GPU

In [ ]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

cantidad_N =   5#@param {type: "number"}
# --------------------------------------------

from datetime import datetime

tiempo_total = datetime.now()

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import numpy

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0


# CPU - Defino la memoria de los vectores en cpu.
x_cpu = numpy.random.randn( cantidad_N )
x_cpu = x_cpu.astype( numpy.float32() )

y_cpu = numpy.random.randn( cantidad_N )
y_cpu = y_cpu.astype( numpy.float32() )

z_cpu = numpy.random.randn( cantidad_N )
z_cpu = y_cpu.astype( numpy.float32() )

v_cpu = numpy.empty_like( x_cpu )

# CPU - reservo la memoria GPU.
x_gpu = cuda.mem_alloc( x_cpu.nbytes )
y_gpu = cuda.mem_alloc( y_cpu.nbytes )
z_gpu = cuda.mem_alloc( z_cpu.nbytes )
v_gpu = cuda.mem_alloc( v_cpu.nbytes )

# GPU - Copio la memoria al GPU.
cuda.memcpy_htod( x_gpu, x_cpu )
cuda.memcpy_htod( y_gpu, y_cpu )
cuda.memcpy_htod( z_gpu, z_cpu )
cuda.memcpy_htod( v_gpu, v_cpu )

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
__global__ void kernel_norm( int n, float *X, float *Y, float *Z, float *V )
{
  int idx = threadIdx.x + blockIdx.x*blockDim.x;
  if( idx < n )
  {
    V[idx] = sqrt(pow(X[idx], 2) + pow(Y[idx], 2) + pow(Z[idx], 2));
  }
}
""")
# CPU - Genero la función kernel.
kernel = module.get_function("kernel_norm")

tiempo_gpu = datetime.now()

# GPU - Ejecuta el kernel.
# TODO: Falta consultar limites del GPU, para armar las dimensiones correctamente.
dim_hilo = 256
dim_bloque = numpy.int( (cantidad_N+dim_hilo-1) / dim_hilo )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

#TODO: Ojo, con los tipos de las variables en el kernel.
kernel( numpy.int32(cantidad_N), x_gpu, y_gpu, z_gpu, v_gpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

tiempo_gpu = datetime.now() - tiempo_gpu

# GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh( v_cpu, v_gpu )

# CPU - Informo el resutlado.
print( "------------------------------------")
print( "X: " )
print( x_cpu )
print( "------------------------------------")
print( "Y: " )
print( y_cpu )
print( "------------------------------------")
print( "Z: " )
print( z_cpu )
print( "------------------------------------")
print( "V: " )
print( v_cpu )

tiempo_total = datetime.now() - tiempo_total

print( "Cantidad de elementos: ", cantidad_N )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )

---
# 4 Tabla de pasos de ejecución del programa


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random.randn( Cantidad_N ) | Inicializa los vectoes A, B y R.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  SourceModule()        | Define el código del kernel 
CPU      |  module.get_function() | Genera la función del kernel GPU
CPU      |  dim_tx/dim_bx         | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU memoria A a CPU memoria R.
CPU      |  print()               | Informo los resultados.



---
# 5 Conclusiones

Las conclusiones son explicadas en clase....


---

# 6 Bibliografia

[1] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Función Axpy de biblioteca BLAS: [Referencia](https://software.intel.com/content/www/us/en/develop/documentation/mkl-developer-reference-c/top/blas-and-sparse-blas-routines/blas-routines/blas-level-1-routines-and-functions/cblas-axpy.html)

[4] Biblioteca BLAS: [Referencia](http://www.netlib.org/blas/)

[5] Documentación PyCUDA: [WEB](https://documen.tician.de/pycuda/index.html)

[6] Repositorio de PyCUDA: [WEB](https://pypi.python.org/pypi/pycuda)


